In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

# Assuming your DataFrame is named df and the text column is 'review_text'
df3 = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/balance_sample_sub.csv')  # Replace with the actual path to your data file
df3 = df3[df3['total_no_drug'].notnull()]


drug_summary = df3.columns.tolist()
count_columns = [col for col in drug_summary if col.startswith('count_')]

# Create the 'labels' column
df3['labels'] = df3.apply(
    lambda row: [col for col in count_columns if row[col] > 0],
    axis=1
)




df3 = df3[df3['match'] == 1]

texts = df3['total_no_drug'].astype(str).tolist()

labels = df3['labels'].tolist()
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

<ipython-input-14-f6a96639e841>:4: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/balance_sample_sub.csv')  # Replace with the actual path to your data file


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.BCEWithLogitsLoss()(outputs, labels)  # Binary Cross-Entropy Loss for multilabel classification

        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    # Convert predictions and actual_labels to multilabel format
    predictions = (np.array(predictions) > 0.5).astype(int)
    actual_labels = np.array(actual_labels)

    # Calculate multilabel metrics
    recall = recall_score(actual_labels, predictions, average='weighted')
    precision = precision_score(actual_labels, predictions, average='weighted')
    f1 = f1_score(actual_labels, predictions, average='weighted')

    return recall, precision, f1

# Load your data (assuming df2 is your DataFrame)
texts = df2['total_no_drug'].astype(str).tolist()

labels = df2['labels'].tolist()
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 8
max_length = 150
batch_size = 100
num_epochs = 4
learning_rate = 2e-5

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer, dataset, and data loader
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Set up the device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training the model
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    recall, precision, f1 = evaluate(model, val_dataloader, device)
    print(f"Validation recall: {recall:.4f}, precision: {precision:.4f}, f1: {f1:.4f}")

# Save the final model
torch.save(model.state_dict(), "bert_classifier_post_type.pth")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Validation recall: 0.7913, precision: 0.8672, f1: 0.8013
Epoch 3/4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation recall: 0.7971, precision: 0.8681, f1: 0.8136
Epoch 4/4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation recall: 0.8057, precision: 0.8664, f1: 0.8192


In [ ]:

torch.save(model.state_dict(), "/content/drive/MyDrive/reddit/bert_classifier_post_type.pth")


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import numpy as np

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 'attention_mask': self.encodings['attention_mask'][idx], 'label': self.labels[idx]}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.BCEWithLogitsLoss()(outputs, labels)  # Binary Cross-Entropy Loss for multilabel classification

        loss.backward()
        optimizer.step()
        scheduler.step()

def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions.extend(torch.sigmoid(outputs).cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    # Convert predictions and actual_labels to multilabel format
    predictions = (np.array(predictions) > 0.5).astype(int)
    actual_labels = np.array(actual_labels)

    # Calculate multilabel metrics
    recall = recall_score(actual_labels, predictions, average='weighted')
    precision = precision_score(actual_labels, predictions, average='weighted')
    f1 = f1_score(actual_labels, predictions, average='weighted')

    return recall, precision, f1

In [ ]:
import torch
from transformers import BertTokenizer
# Make sure to replace 'your_module' with the actual module where your BERTClassifier is defined

# Assuming you have saved the model state_dict as 'bert_classifier.pth'
#import io
#io.BytesIO(uploaded['bert_classifier.pth'])

saved_model_path = 'content/drive/MyDrive/reddit/bert_classifier_post_type.pth'

# Initialize the model architecture
loaded_model = BERTClassifier(bert_model_name='bert-base-uncased', num_classes=8)  # Make sure to provide the correct configuration

# Load the saved state_dict
#loaded_model.load_state_dict(torch.load(io.BytesIO(uploaded['bert_classifier.pth']), map_location=torch.device('cpu')))
loaded_model.load_state_dict(torch.load('content/drive/MyDrive/reddit/bert_classifier_post_type.pth', map_location=torch.device('cpu')))

#loaded_model.load_state_dict(torch.load(saved_model_path))

# Set the model to evaluation mode
loaded_model.eval()

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example text for prediction
example_text = "This is an example text for sentiment analysis."

# Tokenize and prepare the input for the model
encoding = tokenizer(example_text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'content/drive/MyDrive/reddit/bert_classifier_post_type.pth'

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd


In [ ]:
import pandas as pd

# Assuming your DataFrame is named df and the text column is 'review_text'
df = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/no_inc_samp_sub_ready_nn.csv')  # Replace with the actual path to your data file
df2 = df[df['prediction_list'] == 1]




<ipython-input-24-f3602ddb34f4>:4: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/no_inc_samp_sub_ready_nn.csv')  # Replace with the actual path to your data file


In [ ]:
len(df2)

154645

In [ ]:
import torch
from transformers import BertTokenizer

# Assuming you have saved the model state_dict as 'bert_classifier.pth'
saved_model_path = '/content/drive/MyDrive/reddit/bert_classifier_post_type.pth'

# Assuming BERTClassifier is defined in your_module
# Make sure to replace 'your_module' with the actual module where your BERTClassifier is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
# Initialize the model architecture
loaded_model = BERTClassifier(bert_model_name='bert-base-uncased', num_classes=8).to(device)  # Make sure to provide the correct configuration
#loaded_model = BERTClassifier(bert_model_name='bert-base-uncased', num_classes=2)  # Make sure to provide the correct configuration

# Load the saved state_dict
loaded_model.load_state_dict(torch.load(saved_model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

# Set the model to evaluation mode
loaded_model.eval()

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example text for prediction
example_text = "This is an example text for sentiment analysis."

# Tokenize example_text
with torch.no_grad():
    encoding = tokenizer(example_text, return_tensors='pt', max_length=128, padding='max_length', truncation=True).to(device)

# Check GPU availability


# Optimized parameters
batch_size = 2000  # Adjust based on your GPU memory
prediction_list = []
length = 0

# Process data in batches
with torch.no_grad():
    for i in range(0, len(df2['total_text']), batch_size):
        batch_texts = df2['total_text'].iloc[i:i + batch_size].tolist()

        # Tokenize and move data to GPU
        encodings = tokenizer(batch_texts, return_tensors='pt', max_length=128, padding='max_length', truncation=True).to(device)
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']

        # Model inference on GPU
        output = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(output, dim=1)

        # Move predictions back to CPU
        predicted_class = predicted_class.cpu().tolist()

        # Extend prediction_list
        prediction_list.extend(predicted_class)

        length = length + batch_size
        # Print progress
        if (length) % 1000 == 0:
            print(f"Processed {length} instances")

print("Processing complete.")


Using device: cuda
Processed 2000 instances
Processed 4000 instances
Processed 6000 instances
Processed 8000 instances
Processed 10000 instances
Processed 12000 instances
Processed 14000 instances
Processed 16000 instances
Processed 18000 instances
Processed 20000 instances
Processed 22000 instances
Processed 24000 instances
Processed 26000 instances
Processed 28000 instances
Processed 30000 instances
Processed 32000 instances
Processed 34000 instances
Processed 36000 instances
Processed 38000 instances
Processed 40000 instances
Processed 42000 instances
Processed 44000 instances
Processed 46000 instances
Processed 48000 instances
Processed 50000 instances
Processed 52000 instances
Processed 54000 instances
Processed 56000 instances
Processed 58000 instances
Processed 60000 instances
Processed 62000 instances
Processed 64000 instances
Processed 66000 instances
Processed 68000 instances
Processed 70000 instances
Processed 72000 instances
Processed 74000 instances
Processed 76000 instanc

In [ ]:
df2['bert_nn_type'] = prediction_list

<ipython-input-27-a0f04b5b138d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['bert_nn_type'] = prediction_list


In [ ]:
df2

,Unnamed: 0.1,Unnamed: 0,author,id,selftext,subreddit,subreddit_id,title,Country,selftext_NO_PUNC,...,total_text,concat,total_no_drug,year,prediction_nb,xg_boost,prediction_nb_class,xg_boost_multi,prediction_list,bert_nn_type
1,1,1,meteoritee,eiazxp,Happy New Year 2020!Not only do we celebrate t...,CasualUK,t5_3j2jr,Happy New Year 2020!,UK,Happy New Year 2020 Not only do we celebrate t...,...,Happy New Year 2020! Happy New Year 2020!Not o...,[],Happy New Year 2020 Happy New Year 2020 Not on...,2020,1,0,"('count_marijuana',)",NaN,1,3
2,2,2,Medical-Outcome,eib00z,,CasualUK,t5_3j2jr,Starting the next decade in a spoons 👍,UK,,...,Starting the next decade in a spoons 👍,[],Starting the next decade in a spoons 👍,2020,0,0,NaN,NaN,1,6
10,10,10,brunofin,eib1sa,,poland,t5_2qkmn,Happy new year from poznań,Poland,,...,Happy new year from poznań,[],Happy new year from poznań,2020,0,0,NaN,NaN,1,3
11,11,11,Dragontriplo,eib2m6,,portugal,t5_2qmie,Feliz 2020 pessoal,Portugal,,...,Feliz 2020 pessoal,[],Feliz 2020 pessoal,2020,0,0,NaN,NaN,1,6
13,13,13,DaSaYaMa,eib3vb,"Bei uns in der Straße, Anliegerstraße und sehr...",de,t5_22i0,Ich hatte meinen deutschesten Moment des Jahre...,Germany,Bei uns in der Straße Anliegerstraße und sehr...,...,Ich hatte meinen deutschesten Moment des Jahre...,[],Ich hatte meinen deutschesten Moment des Jahre...,2020,1,1,"('count_marijuana',)","('count_marijuana',)",1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326794,1326794,1326794,AfraidFerret2605,16wl0m1,Ich habe mir schon 3 Mal Schuhe bei Zara gekau...,germany,t5_2qi4z,Zara Schuhe piepen,Germany,Ich habe mir schon 3 Mal Schuhe bei Zara gekau...,...,Zara Schuhe piepen Ich habe mir schon 3 Mal Sc...,[],Zara Schuhe piepen Ich habe mir schon 3 Mal Sc...,2023,1,1,"('count_marijuana',)","('count_marijuana',)",1,6
1326796,1326796,1326796,FerretEuphoric9992,16wl58g,Ich habe kürzlich im Internet diese Website ei...,germantrees,t5_2tcsn,Weed aus dem Clear web?,germany,Ich habe kürzlich im Internet diese Website ei...,...,Weed aus dem Clear web? Ich habe kürzlich im I...,['weed'],aus dem Clear web Ich habe kürzlich im Interne...,2023,1,1,"('count_marijuana',)","('count_marijuana',)",1,6
1326797,1326797,1326797,cyberoscar,16wl6aq,Jag inser att detta bara är en i mängden posts...,sweden,t5_2qofe,Min Take,Sweden,Jag inser att detta bara är en i mängden posts...,...,Min Take Jag inser att detta bara är en i mäng...,['cannabis'],Min Take Jag inser att detta bara är en i mäng...,2023,0,1,NaN,"('count_fentanyl', 'count_heroin', 'count_mari...",1,6
1326804,1326804,1326804,Zattlaegott,16wlg27,hej jag och en polare är extremt sugna på att ...,droger,t5_r5ugy,Hej någon som har har testat dmt och kan berät...,danish,hej jag och en polare är extremt sugna på att ...,...,Hej någon som har har testat dmt och kan berät...,[],Hej någon som har har testat dmt och kan berät...,2023,1,1,"('count_marijuana',)","('count_fentanyl', 'count_heroin', 'count_mari...",1,6


In [ ]:
sorted(count_columns)

['count_COCAINE',
 'count_GHB',
 'count_INHALANTS',
 'count_MDMA',
 'count_fentanyl',
 'count_heroin',
 'count_marijuana',
 'count_oxy']

In [ ]:
df2['bert_nn_class'] = df2['bert_nn_type'].map(lambda x: sorted(count_columns)[x])
df2

<ipython-input-30-68d2d37736e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['bert_nn_class'] = df2['bert_nn_type'].map(lambda x: sorted(count_columns)[x])


,Unnamed: 0.1,Unnamed: 0,author,id,selftext,subreddit,subreddit_id,title,Country,selftext_NO_PUNC,...,total_no_drug,year,prediction_nb,xg_boost,prediction_nb_class,xg_boost_multi,prediction_list,bert_nn_type,item_column,bert_nn_class
1,1,1,meteoritee,eiazxp,Happy New Year 2020!Not only do we celebrate t...,CasualUK,t5_3j2jr,Happy New Year 2020!,UK,Happy New Year 2020 Not only do we celebrate t...,...,Happy New Year 2020 Happy New Year 2020 Not on...,2020,1,0,"('count_marijuana',)",NaN,1,3,count_MDMA,count_MDMA
2,2,2,Medical-Outcome,eib00z,,CasualUK,t5_3j2jr,Starting the next decade in a spoons 👍,UK,,...,Starting the next decade in a spoons 👍,2020,0,0,NaN,NaN,1,6,count_marijuana,count_marijuana
10,10,10,brunofin,eib1sa,,poland,t5_2qkmn,Happy new year from poznań,Poland,,...,Happy new year from poznań,2020,0,0,NaN,NaN,1,3,count_MDMA,count_MDMA
11,11,11,Dragontriplo,eib2m6,,portugal,t5_2qmie,Feliz 2020 pessoal,Portugal,,...,Feliz 2020 pessoal,2020,0,0,NaN,NaN,1,6,count_marijuana,count_marijuana
13,13,13,DaSaYaMa,eib3vb,"Bei uns in der Straße, Anliegerstraße und sehr...",de,t5_22i0,Ich hatte meinen deutschesten Moment des Jahre...,Germany,Bei uns in der Straße Anliegerstraße und sehr...,...,Ich hatte meinen deutschesten Moment des Jahre...,2020,1,1,"('count_marijuana',)","('count_marijuana',)",1,6,count_marijuana,count_marijuana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326794,1326794,1326794,AfraidFerret2605,16wl0m1,Ich habe mir schon 3 Mal Schuhe bei Zara gekau...,germany,t5_2qi4z,Zara Schuhe piepen,Germany,Ich habe mir schon 3 Mal Schuhe bei Zara gekau...,...,Zara Schuhe piepen Ich habe mir schon 3 Mal Sc...,2023,1,1,"('count_marijuana',)","('count_marijuana',)",1,6,count_marijuana,count_marijuana
1326796,1326796,1326796,FerretEuphoric9992,16wl58g,Ich habe kürzlich im Internet diese Website ei...,germantrees,t5_2tcsn,Weed aus dem Clear web?,germany,Ich habe kürzlich im Internet diese Website ei...,...,aus dem Clear web Ich habe kürzlich im Interne...,2023,1,1,"('count_marijuana',)","('count_marijuana',)",1,6,count_marijuana,count_marijuana
1326797,1326797,1326797,cyberoscar,16wl6aq,Jag inser att detta bara är en i mängden posts...,sweden,t5_2qofe,Min Take,Sweden,Jag inser att detta bara är en i mängden posts...,...,Min Take Jag inser att detta bara är en i mäng...,2023,0,1,NaN,"('count_fentanyl', 'count_heroin', 'count_mari...",1,6,count_marijuana,count_marijuana
1326804,1326804,1326804,Zattlaegott,16wlg27,hej jag och en polare är extremt sugna på att ...,droger,t5_r5ugy,Hej någon som har har testat dmt och kan berät...,danish,hej jag och en polare är extremt sugna på att ...,...,Hej någon som har har testat dmt och kan berät...,2023,1,1,"('count_marijuana',)","('count_fentanyl', 'count_heroin', 'count_mari...",1,6,count_marijuana,count_marijuana


In [ ]:
df3 = pd.merge(df,df2[['id','bert_nn_class']], how = 'left', on = 'id')
df3

,Unnamed: 0.1,Unnamed: 0,author,id,selftext,subreddit,subreddit_id,title,Country,selftext_NO_PUNC,...,total_text,concat,total_no_drug,year,prediction_nb,xg_boost,prediction_nb_class,xg_boost_multi,prediction_list,bert_nn_class
0,0,0,WonderfulDog69,eiazwx,,portugal,t5_2qmie,"Primeiro vídeo do ano, bora. Um enorme 2020 a ...",Portugal,,...,"Primeiro vídeo do ano, bora. Um enorme 2020 a ...",[],Primeiro vídeo do ano bora Um enorme 2020 a to...,2020,0,0,NaN,NaN,0,NaN
1,1,1,meteoritee,eiazxp,Happy New Year 2020!Not only do we celebrate t...,CasualUK,t5_3j2jr,Happy New Year 2020!,UK,Happy New Year 2020 Not only do we celebrate t...,...,Happy New Year 2020! Happy New Year 2020!Not o...,[],Happy New Year 2020 Happy New Year 2020 Not on...,2020,1,0,"('count_marijuana',)",NaN,1,count_MDMA
2,2,2,Medical-Outcome,eib00z,,CasualUK,t5_3j2jr,Starting the next decade in a spoons 👍,UK,,...,Starting the next decade in a spoons 👍,[],Starting the next decade in a spoons 👍,2020,0,0,NaN,NaN,1,count_marijuana
3,3,3,Rizlem,eib099,"I hope you have a happy new year, and your nex...",unitedkingdom,t5_2qhqb,HAPPY NEW YEAR!,UK,I hope you have a happy new year and your nex...,...,HAPPY NEW YEAR! I hope you have a happy new ye...,[],HAPPY NEW YEAR I hope you have a happy new yea...,2020,1,0,"('count_marijuana',)",NaN,0,NaN
4,4,4,Arculae,eib0d0,,uktrees,t5_2si9d,Happy New Year,uk,,...,Happy New Year,[],Happy New Year,2020,1,0,"('count_marijuana',)",NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326814,1326814,1326814,Julial12,16wlxmr,,CasualUK,t5_3j2jr,Home alone here 😪,UK,,...,Home alone here 😪,[],Home alone here 😪,2023,1,0,"('count_marijuana',)",NaN,0,NaN
1326815,1326815,1326815,Allmychickenbois,16wlxow,"(SFW stuff only, you animals!)You know those n...",CasualUK,t5_3j2jr,What do you do to help yourself fall asleep?,UK,SFW stuff only you animals You know those ni...,...,What do you do to help yourself fall asleep? (...,[],What do you do to help yourself fall asleep SF...,2023,1,0,"('count_marijuana',)",NaN,0,NaN
1326816,1326816,1326816,Free_Price3574,16wlyso,"Hej!Jag har en situation som precis hände, och...",sweden,t5_2qofe,Granne (Ordförande för BRFen) gick in i min lä...,Sweden,Hej Jag har en situation som precis hände och...,...,Granne (Ordförande för BRFen) gick in i min lä...,[],Granne Ordförande för BRFen gick in i min läge...,2023,0,0,NaN,NaN,0,NaN
1326817,1326817,1326817,Remarkable_Put_7952,16wm2d8,,thenetherlands,t5_30hrx,Den Haag (1929),netherlands,,...,Den Haag (1929),[],Den Haag 1929,2023,0,0,NaN,NaN,0,NaN


In [ ]:
df3.to_csv('/content/drive/MyDrive/reddit/balanced_submission/no_include_sample_sub_nb_xg_bert.csv', index=False)


